In [1]:
import pandas as pd

# Maintenance CSV
maintenance_data = [
    [1,"2025-08-01","Engine A","Oil leakage","Replace oil seal and check pressure"],
    [2,"2025-08-02","Landing Gear B","Hydraulic failure","Inspect hydraulic lines and refill fluid"],
    [3,"2025-08-03","Fuselage C","Corrosion","Apply anti-corrosion treatment and inspect panels"],
    [4,"2025-08-04","Avionics D","Navigation error","Run diagnostic and update firmware"],
    [5,"2025-08-05","Engine E","Overheating","Replace thermostat and clean cooling system"]
]
df_maintenance = pd.DataFrame(maintenance_data, columns=["id","date","equipment","issue","procedure"])
df_maintenance.to_csv("maintenance.csv", index=False)

# Aircraft Taxonomy CSV
taxonomy_data = [
    [1,"Engine","Turbofan","High-bypass turbofan engine used in commercial jets"],
    [2,"Landing Gear","Main Gear","Retractable main landing gear assembly"],
    [3,"Fuselage","Body","Pressurized aircraft body structure"],
    [4,"Avionics","Navigation","Navigation system including GPS and INS"],
    [5,"Electrical","Power","Electrical power distribution system"],
    [6,"Hydraulics","Actuation","Hydraulic system controlling flaps and landing gear"],
    [7,"Fuel","Storage","Fuel tanks and fuel distribution system"],
    [8,"Cabin","Seats","Passenger seating and cabin layout"],
    [9,"Flight Controls","Elevators","Primary control surfaces for pitch"],
    [10,"Flight Controls","Ailerons","Primary control surfaces for roll"]
]
df_taxonomy = pd.DataFrame(taxonomy_data, columns=["id","category","subcategory","description"])
df_taxonomy.to_csv("aircrafttaxonomy.csv", index=False)

In [ ]:
## Ingestion pipeline to load data
import os
import json
import pandas as pd
import requests
import httpx
from sqlalchemy import create_engine, text
from langchain.docstore.document import Document
from langchain_postgres.vectorstores import PGVector

# -----------------------------
# Load services from env
# -----------------------------
vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

# -----------------------------
# Embedding service details
# -----------------------------
def is_embeddingservice(service):
    return service["name"] == "prod-embedding-nomic-text"

embedding_services = filter(is_embeddingservice, services["genai"])
embedding_credentials = list(embedding_services)[0]["credentials"]

api_base = embedding_credentials["api_base"] + "/v1"
api_key = embedding_credentials["api_key"]
model_name = embedding_credentials["model_name"]

print("Embedding model:", model_name)

# -----------------------------
# Database connection
# -----------------------------
def is_vectordbservice(service):
    return service["name"] == "vector-db"

db_services = filter(is_vectordbservice, services["postgres"])
db_credentials = list(db_services)[0]["credentials"]
db_uri = db_credentials["uri"]

print("DB URI:", db_uri)

engine = create_engine(db_uri)

# Test DB connection
with engine.connect() as conn:
    version = conn.execute(text("SELECT version();")).fetchone()
    print("Connected to:", version[0])

# -----------------------------
# Embedding function (REST call)
# -----------------------------
url = api_base + "/embeddings"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

def embed_text(text: str):
    payload = {"model": "nomic-embed-text", "input": text}
    resp = requests.post(url, headers=headers, json=payload, verify=False)
    resp.raise_for_status()
    return resp.json()["data"][0]["embedding"]

class CustomEmbeddings:
    def embed_documents(self, texts): return [embed_text(t) for t in texts]
    def embed_query(self, text): return embed_text(text)

embedding = CustomEmbeddings()

# -----------------------------
# PGVector setup
# -----------------------------
vectorstore = PGVector(
    embeddings=embedding,
    connection=db_uri,
    collection_name="maintenance_and_taxonomy",
    use_jsonb=True,
    create_extension=True,       # will create pgvector extension if not exists
    pre_delete_collection=True,  # clears old data on restart
)

# -----------------------------
# Load maintenance.csv
# -----------------------------
def sanitize_metadata(metadata):
    sanitized = {}
    for k, v in metadata.items():
        if isinstance(v, set):
            sanitized[k] = list(v)
        elif not isinstance(v, (str, int, float, bool, dict, list, type(None))):
            sanitized[k] = str(v)
        else:
            sanitized[k] = v
    return sanitized

df_maintenance = pd.read_csv("maintenance.csv")  # columns: id, date, equipment, issue, procedure

docs_csv = [
    Document(
        page_content=f"{row['equipment']}: {row['issue']} - {row['procedure']}",
        metadata=sanitize_metadata({"id": row["id"], "source": "maintenance.csv"})
    )
    for _, row in df_maintenance.iterrows()
]

# -----------------------------
# Load aircrafttaxonomy.csv
# -----------------------------
df_taxonomy = pd.read_csv("aircrafttaxonomy.csv")  # columns: id, category, subcategory, description

docs_taxonomy = [
    Document(
        page_content=f"{row['category']} / {row['subcategory']}: {row['description']}",
        metadata=sanitize_metadata({"id": row["id"], "source": "aircrafttaxonomy.csv"})
    )
    for _, row in df_taxonomy.iterrows()
]

# -----------------------------
# Insert into vectorstore
# -----------------------------
all_docs = docs_csv + docs_taxonomy
vectorstore.add_documents(all_docs)

print(f"✅ Inserted {len(all_docs)} documents into the vectorstore!")

# -----------------------------
# Inspect DB
# -----------------------------
query = text("SELECT * FROM langchain_pg_collection LIMIT 5;")
print(pd.read_sql(query, engine))

query2 = text("SELECT * FROM langchain_pg_embedding LIMIT 5;")
print(pd.read_sql(query2, engine))

In [3]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_openai import OpenAIEmbeddings
from datetime import date
import warnings
import ssl
from langchain_community.embeddings import OllamaEmbeddings
from openai import OpenAI
from langchain.chains import RetrievalQA


httpx_client = httpx.Client(http2=True, verify=False, timeout=30.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]


llm = ChatOpenAI(temperature=0.9, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)

# Create a retriever from your vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

# Build a RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Ask a question
query = "Which aircraft equipment has reported issues with hydraulic leaks?"
result = qa.run(query)
print(result)

/home/vcap/tmp/ipykernel_6122/3516390421.py:48: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)
/home/vcap/app/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-proxy.sys.tas-ndc.kuhn-labs.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<think>
Okay, let's see. The user is asking which aircraft equipment has reported issues with hydraulic leaks. The context provided mentions "Landing Gear B: Hydraulic failure - Inspect hydraulic lines and refill fluid" and "Hydraulics / Actuation: Hydraulic system controlling flaps and landing gear." 

So, the user's question is about equipment that had hydraulic leaks. The context specifically points to the landing gear's hydraulic system. The entry under "Landing Gear B" mentions hydraulic failure, which involves inspecting hydraulic lines and refilling fluid. Additionally, the hydraulics/actuation section mentions the system controlling flaps and landing gear. 

Wait, but the question is about hydraulic leaks. The context provided doesn't explicitly mention leaks, but rather hydraulic failure, which could include leaks. However, the answer might be the landing gear's hydraulic system. The user might be referring to the equipment listed in the context. Since the context only mention

In [13]:
import httpx
from openai import OpenAI as RawOpenAI
from trulens.providers.openai import OpenAI as TruOpenAI

class CustomOpenAI(TruOpenAI):
    def __init__(self, model_engine: str, api_key: str, api_base: str, verify: bool = True):
        # Create custom HTTP client (disable SSL if verify=False)
        http_client = httpx.Client(http2=True, verify=verify, timeout=60.0)

        # Wrap inside the official OpenAI client (valid for TruLens)
        raw_client = RawOpenAI(
            api_key=api_key,
            base_url=api_base,
            http_client=http_client
        )

        # Initialize TruOpenAI with proper OpenAI client
        super().__init__(
            model_engine=model_engine,
            client=raw_client
        )

        # Store extras bypassing Pydantic setattr restrictions
        object.__setattr__(self, "raw_client", raw_client)
        object.__setattr__(self, "_http_client", http_client)
        object.__setattr__(self, "_model_engine", model_engine)

    # Direct API helpers
    def chat(self, messages, model=None, **kwargs):
        model = model or self._model_engine
        return self.raw_client.chat.completions.create(
            model=model,
            messages=messages,
            **kwargs
        )

    def embed(self, input, model=None, **kwargs):
        model = model or self._model_engine
        return self.raw_client.embeddings.create(
            model=model,
            input=input,
            **kwargs
        )


In [21]:
import httpx
import json


# Your custom endpoint
url = chat_credentials["api_base"] + "/v1/responses"

# Your API key (or token)
api_key = chat_credentials["api_key"]

# Example payload
payload = {
    "model": chat_credentials["model_name"],
    "input": "Which aircraft equipment had hydraulic issues?",
    "temperature": 0
}

# Create a client that bypasses SSL verification
with httpx.Client(verify=False, http2=True, timeout=60.0) as client:
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    response = client.post(url, headers=headers, data=json.dumps(payload))

# Print status and response
print("Status code:", response.status_code)
try:
    print("Response JSON:", response.json())
except Exception:
    print("Raw response:", response.text)

Status code: 404
Response JSON: {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}


In [15]:
# uv add trulens-core trulens-apps-langchain trulens-providers-openai
import numpy as np
from trulens.apps.langchain import TruChain
from trulens.core import Feedback, TruSession, Tru
from trulens.providers.openai import OpenAI

import os
import httpx

from trulens.providers.openai.endpoint import OpenAIEndpoint

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

# Specify your local endpoint URL and API key
custom_api_base = chat_credentials["api_base"] + "/v1"
custom_api_key = chat_credentials["api_key"]
custom_model_name = chat_credentials["model_name"]

# Create a custom HTTP client with SSL verification disabled
custom_http_client = httpx.Client(verify=False)


# Create the TruLens OpenAI provider with the custom endpoint
provider = CustomOpenAI(
    model_engine=custom_model_name,
    api_key=custom_api_key,
    api_base=custom_api_base,
    verify=False
)
# Initialize session
tru = Tru()


context = TruChain.select_context(qa)

f_context_relevance = (
    Feedback(provider.context_relevance)
    .on_input()
    .on_context(collect_list=False)
    .aggregate(np.mean)
)

# Wrap and evaluate
tru_recorder = TruChain(
    qa,
    app_name="Aircraft Maintenance Chatbot",
    app_id="app_aircraft_maintenance",
    app_version="v1",
    feedbacks=[f_context_relevance],
)


# Example query
query = "Which aircraft equipment had hydraulic issues?"

with tru_recorder as recording:
    response = qa(query)
    print("Context relevance score:", response)
    
records, feedback = tru.get_records_and_feedback(app_ids=[])
print(records.head())

tru.get_leaderboard(app_ids=[])
# Launch dashboard
tru.run_dashboard(        
    address="0.0.0.0",  # bind to all network interfaces for external access
    port=7861,       # port to run the dashboard
)


/home/vcap/tmp/ipykernel_6122/458173627.py:38: DeprecationWarning: Tru is deprecated, use TruSession instead.
  tru = Tru()


instrumenting <class 'langchain_core.prompts.chat.ChatPromptTemplate'> for base <class 'langchain_core.prompts.chat.ChatPromptTemplate'>
	instrumenting invoke
	instrumenting ainvoke
	instrumenting stream
	instrumenting astream
instrumenting <class 'langchain_core.prompts.chat.ChatPromptTemplate'> for base <class 'langchain_core.prompts.chat.BaseChatPromptTemplate'>
	instrumenting invoke
	instrumenting ainvoke
	instrumenting stream
	instrumenting astream
instrumenting <class 'langchain_core.prompts.chat.ChatPromptTemplate'> for base <class 'langchain_core.prompts.base.BasePromptTemplate'>
	instrumenting invoke
	instrumenting ainvoke
	instrumenting stream
	instrumenting astream
instrumenting <class 'langchain_core.prompts.chat.ChatPromptTemplate'> for base <class 'langchain_core.runnables.base.RunnableSerializable[dict, PromptValue]'>
	instrumenting invoke
	instrumenting ainvoke
	instrumenting stream
	instrumenting astream
instrumenting <class 'langchain_core.prompts.chat.ChatPromptTempl

/home/vcap/app/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-proxy.sys.tas-ndc.kuhn-labs.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=3.
OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=2.
OpenAIEndpo

Context relevance score: {'query': 'Which aircraft equipment had hydraulic issues?', 'result': '<think>\nOkay, the user is asking which aircraft equipment had hydraulic issues. Let me check the context provided.\n\nThe context mentions "Landing Gear B: Hydraulic failure - Inspect hydraulic lines and refill fluid" and "Hydraulics / Actuation: Hydraulic system controlling flaps and landing gear." So, the hydraulic system is related to both the landing gear and the flaps. The specific equipment mentioned here are the landing gear and the flaps, as they are controlled by the hydraulic system. The problem stated is a hydraulic failure in the landing gear, so the answer should be the landing gear. But also, since the hydraulic system controls both flaps and landing gear, maybe the answer includes both? Wait, the user is asking which equipment had hydraulic issues. The context says "Hydraulic failure" under Landing Gear B, so the specific equipment is the landing gear. The hydraulics/actuatio

Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=0.
feedback_name=context_relevance, record=659546c2-9ab0-4ffd-ba76-f2270454df0f, span_group=None had an error during computation:
Endpoint OpenAIEndpoint request failed 4 time(s): 
	Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}
	Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}
	Error code: 404 

RuntimeError: Dashboard failed to start in time. Please inspect dashboard logs for additional information.

OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=2.
OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=1.
OpenAIEndpoint request failed <class 'openai.NotFoundError'>=Error code: 404 - {'type': 'about:blank', 'title': 'Not Found', 'status': 404, 'detail': 'No static resource prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses.', 'instance': '/prod-chat-tools-qwen3-ultra-48b5cd6/openai/v1/responses'}. Retries remaining=0.
feedback_name=context_relevance